#### This code Tests following configuration:

Entity Representation: Words in Entity and Context Words

Type Representation: Similar words to type representation learned on training corpus

Ranking Score: Group Average

Word Representation: Trained word vectors on corpus initialized with pretrained glove vector embeddings

In [1]:
import os, logging, pickle
from operator import itemgetter
import numpy as np
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
entity_ranking_op_folder = '../../Data/output'
type_rep_folder = 'TypeRep/type_word/train'
entity_rep_test_folder = 'EntityRep/test'
entity_context_word_file = 'doc_entity_context_word.p'
entity_doc_level_context_word_file = 'doc_entity_doc_level_context_word.p'
type_word_file_name = 'tag_word_dict.p'

doc_level = True
sent_level = False

word_emnbedding_pretrained_trained_on_corpus = '../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt'

entity_context_word_file = os.path.join(entity_ranking_op_folder, entity_rep_test_folder, entity_context_word_file)
entity_doc_level_context_word_file = os.path.join(entity_ranking_op_folder, entity_rep_test_folder, entity_doc_level_context_word_file)
type_word_file = os.path.join(entity_ranking_op_folder, type_rep_folder, type_word_file_name)

In [3]:
tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others', 'ORG_Accused', 'ORG_Victim', 'ORG_Others', 'PER_Victim', 'PER_Others', 'PER_Accused']
per_tag_list = ['PER_Victim', 'PER_Others', 'PER_Accused']
loc_tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others']
org_tag_list = ['ORG_Accused', 'ORG_Victim', 'ORG_Others']

In [5]:
if sent_level:
    entity_rep_context_word_file_dict = pickle.load(open(entity_context_word_file, 'rb'))
if doc_level:
    entity_rep_context_word_file_dict = pickle.load(open(entity_doc_level_context_word_file, 'rb'))
type_word_dict = pickle.load(open(type_word_file, 'rb'))
for tag in tag_list:
    print(tag, type_word_dict[tag], '\n')

LOC_Event [('The', 0.8279561996459961), ('the', 0.8051686882972717), (',', 0.7884728908538818), ('blast', 0.7716223001480103), ('bomb', 0.7610410451889038), ('Friday', 0.7547277212142944), ('blasts', 0.747754693031311), ('in', 0.7384704351425171), ('a', 0.7354503870010376), ('of', 0.7251454591751099), ('.', 0.7246488928794861), ('at', 0.7192941308021545), ('people', 0.7191165685653687), ("'s", 0.715711236000061), ('was', 0.715296745300293), ('one', 0.7113204002380371), ('Park', 0.7099930047988892), ('from', 0.7097682356834412), ('Swami', 0.708998441696167), ('on', 0.7053173780441284), ('this', 0.7003207206726074), ('Police', 0.6995710730552673), ('India', 0.6982031464576721), ('Mr.', 0.6977953910827637), ('killed', 0.6958668231964111), ('Reddy', 0.6933578252792358), ('city', 0.6923993825912476), ('bombing', 0.6901836395263672), ('also', 0.689039945602417), ('and', 0.6885293126106262)] 

LOC_Accused [('The', 0.7958940863609314), (',', 0.7449305057525635), ('several', 0.74460369348526), 

In [6]:
word_vectors = KeyedVectors.load_word2vec_format(word_emnbedding_pretrained_trained_on_corpus, binary=False)
word_vector = word_vectors.get_vector('Narendra')
size = len(word_vector)

2018-05-19 01:29:01,327 : INFO : loading projection weights from ../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt
2018-05-19 01:31:25,073 : INFO : loaded (408422, 300) matrix from ../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt


In [7]:
def group_average(entity, role_tuple):
    num_of_words_in_entity = len(entity)
    num_of_words_in_role = len(role)
    total_number_of_words = num_of_words_in_entity + num_of_words_in_role
    
    sum_of_entity_type_word_vec = np.zeros(size)
    for word in entity:
        try:
            word_vec = word_vectors.get_vector(word)
            word_vec = word_vec/np.linalg.norm(word_vec)
        except KeyError:
            word_vec = np.zeros(size)
            
        sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, word_vec)
    
    for tup in role_tuple:
        try:
            word_vec = word_vectors.get_vector(tup[0])
            word_vec = word_vec/np.linalg.norm(word_vec)
        except KeyError:
            word_vec = np.zeros(size)
        
        sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, word_vec)
        
    dot_product = np.dot(sum_of_entity_type_word_vec, sum_of_entity_type_word_vec)
    group_avg = float(dot_product - (total_number_of_words))/(total_number_of_words * (total_number_of_words - 1))
    return group_avg    
    

In [14]:
precision1 = dict()
doc_count_for_tag = dict()
AveP = dict()

for tag in tag_list:
    doc_count_for_tag[tag] = 0
    precision1[tag] = 0
    AveP[tag] = 0
    

# Precision at K
K = 5

for doc_id in entity_rep_context_word_file_dict:
    #print(doc_id)
    num_actual_entities_with_role = dict()
   
    per_entities = list()
    loc_entities = list()
    org_entities = list()
    doc_role_dict = entity_rep_context_word_file_dict[doc_id]
    for role in doc_role_dict.keys():
        entities = doc_role_dict[role]
        num_actual_entities_with_role[role] = len(entities)
        if role in per_tag_list:
            for entity in entities:
                per_entities.append((entity, role))
        if role in org_tag_list:
            for entity in entities:
                org_entities.append((entity, role))
        if role in loc_tag_list:
            for entity in entities:
                loc_entities.append((entity, role))
    
    num_per_entity = len(per_entities)
    num_org_entity = len(org_entities)
    num_loc_entity = len(loc_entities)
    
    for role in per_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            avp = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in per_entities:
                sim = 2
                sim = group_average(entity[0][1], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_per_entity)):
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
                    avp = avp + (float(TP)/(i+1))
            #if TP ==  num_actual_entities_with_role[role]:
             #   precision1[role] = precision1[role] + 1
            #else:
            precision1[role] = precision1[role] + float(TP)/min(K, num_per_entity)
            if TP != 0:
                AveP[role] = AveP[role] + float(avp)/TP
                
    for role in loc_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            avp = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in loc_entities:
                sim = 2
                sim = group_average(entity[0][1], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_loc_entity)):
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
                    avp = avp + (float(TP)/(i+1))
            #if TP ==  num_actual_entities_with_role[role]:
             #   precision1[role] = precision1[role] + 1
            #else:
            precision1[role] = precision1[role] + float(TP)/min(K, num_loc_entity)
            if TP != 0:
                AveP[role] = AveP[role] + float(avp)/TP
                
    for role in org_tag_list:
        role_word = type_word_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            avp = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in org_entities:
                sim = 2
                sim = group_average(entity[0][1], role_word)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_org_entity)):
                
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
                    avp = avp + (float(TP)/(i+1))
           # if TP ==  num_actual_entities_with_role[role]:
               # precision1[role] = precision1[role] + 1
            #else:
            precision1[role] = precision1[role] + float(TP)/min(K, num_org_entity)
            if TP != 0:
                AveP[role] = AveP[role] + float(avp)/TP
    #print(num_actual_entities_with_role)
    #count = 1

avg_pre = 0
mean_avg_pre = 0
for tag in tag_list:
    if doc_count_for_tag[tag] > 0:
        avg_pre = avg_pre + float(precision1[tag] * 100)/doc_count_for_tag[tag]
        mean_avg_pre = mean_avg_pre + float(AveP[tag] * 100)/doc_count_for_tag[tag]
        #print(tag, float(precision1[tag] * 100)/doc_count_for_tag[tag], '%')
        print(tag, float(AveP[tag] * 100)/doc_count_for_tag[tag], '%')
#print('Average', avg_pre/10, '%')
print('Mean Average Precision', mean_avg_pre/10, '%')

LOC_Event 58.244756235827666 %
LOC_Accused 51.51643990929705 %
LOC_Victim 48.14814814814815 %
LOC_Others 76.10778443113769 %
ORG_Accused 59.341309130913125 %
ORG_Victim 47.849616858237546 %
ORG_Others 88.78272669220944 %
PER_Victim 54.62365591397849 %
PER_Others 84.20206971677558 %
PER_Accused 77.54185692541859 %
Mean Average Precision 64.63583639619432 %
